# Gas Station Optimization Problem
#### _Edward Krueger, edkrueger@gmail.com_

## Purpose

Learn how to modularize a simulation so that we can seach over different inputs to find an optimum.

## Overview

Now that you have completed Alice's simulation, she want you to answer her over all question: How many pumps should she have at her gas station. She wants to know how many pumps are needed to maximize gallons sold.  In order to do this we will impove on the code from the Gas Station Simulation Problem. Feel free to use either your own code or the code in the solution as a starting point. 

## The details

### Encapsulate the Simulation

We need to rewrite the simulation as a encapsulated function that takes every varible the simulation needs to run as an argument. This will require that for every function or object, the parameters are passed explicitly as arguments; rather than being read as global varialbes.

This will allow us to easily see the effect of gas pumps on the number of gallons sold.

Tips:
* We can either eliminate the print statement of include a flag telling whether or not the simulation should print results. The simulation and optimization will run much faster if they don't have to print and the output will be much cleaner.
* Since we are only focused on the number of gallons sold, we don't need the other metrics. Since the GasStation instance keeps track of all of the gas sold, we don't need to save the Car instances in a list.

### Simulate for Different Numbers of Pumps

Now since we have a function that returns the simulated number of gallons sold, we can to run this function for a number of values to see its results.

# Gas Station Optimization Solution

## Setup

In [1]:
import simpy

import numpy as np
import pandas as pd

from scipy.stats import expon
from scipy.stats import norm

In [2]:
class GasStation():
    
    """
    Models a gas station as a simpy.Resource with a given number of pumps.
    
    Arguments:
    -----------
    num_pumps (int): The number of pumps at the gas station
    env (simpy.Environment): The simpy environment
    """
    
    def __init__(self, num_pumps, env):
        
        self.num_pumps = num_pumps
        
        self.env = env
        
        self.resource = simpy.Resource(self.env, capacity=num_pumps)
        
        self.minutes_utilized = 0
        self.gallons_sold = 0

In [3]:
class Car():

    """
    Models a car.
    The car's simulation process in the simulation is the method .run().
    
    Arguments:
    -----------
    id (int): A unique id for the car
    gas_required (float): Amount of gallons a car will fill
    lay_time (float): Extra minutes that the car stays at the pump
    gas_station (GasStation): The gas station the cars are going to
    env (simpy.Environment): The simpy environment
    
    """
    
    def __init__(
        self,
        id,
        gas_required,
        lay_time,
        gas_station,
        pump_rate,
        env,
        verbose
    ):
        
        self.id = id
        self.gas_required = gas_required
        self.lay_time = lay_time
        self.gas_station = gas_station
        self.pump_rate = pump_rate
        self.env = env
        
        self.verbose = verbose
        
        self.wait_time = np.inf
        self.finished = False
        
        self.action = self.env.process(self.run())
        
    def run(self):
        
        queue_time = self.env.now
        
        if self.verbose:
            print(f'Car {self.id} arrives at {round(queue_time, 2)} minutes')
                
        with self.gas_station.resource.request() as req:
            
            yield req
            
            pump_start_time = self.env.now
            
            if self.verbose:
                print(f'Car {self.id} begins utilizing a pump at {round(pump_start_time, 2)} minutes')

            pump_time = self.gas_required / self.pump_rate
            utilization = max(pump_time, self.lay_time)
            
            yield self.env.timeout(utilization)
            
            pump_end_time = self.env.now
            
            if self.verbose:
                print(f'Car {self.id} leaves its pump at {round(pump_end_time, 2)} minutes')
            
            self.finished = True
            
            self.wait_time = pump_start_time - queue_time
            self.gas_station.minutes_utilized += utilization
            self.gas_station.gallons_sold += self.gas_required

In [4]:
def scheduler(
    expected_wait,
    gas_required_mean,
    gas_required_std,
    pump_rate,
    expected_lay_time,
    gas_station,
    cars,
    env,
    verbose
):
    
    """
    Controls the arrival of cars at the gas station.
    
    Arguments:
    -----------
    env (simpy.Environment): The simpy environment
    """
    
    id = 0
    
    while True:
        
        waiting_time = expon.rvs(loc=0, scale=expected_wait)
        
        std_norm = norm.rvs()
        gas_required = std_norm * gas_required_std + gas_required_mean
        gas_required = max([0, gas_required])
        
        lay_time = expon.rvs(loc=0, scale=expected_lay_time)
        
        yield env.timeout(waiting_time)
        
        cars.append(
            Car(
                id=id,
                gas_required=gas_required,
                lay_time=lay_time,
                gas_station=gas_station,
                pump_rate = pump_rate,
                env=env,
                verbose=verbose
            )
        )
        
        id += 1

In [ ]:
def sim(
    num_pumps,
    expected_wait,
    gas_required_mean,
    gas_required_std,
    pump_rate,
    expected_lay_time,
    sim_time,
    verbose = False
):
    
    env = simpy.Environment()
    cars = []
    gas_station = GasStation(num_pumps, env)
    env.process(
        scheduler(
            expected_wait,
            gas_required_mean,
            gas_required_std,
            pump_rate,
            expected_lay_time,
            gas_station,
            cars,
            env,
            verbose=verbose
        )
    )
    env.run(until=sim_time)
    
    total_gallons_sold = gas_station.gallons_sold
    
    return total_gallons_sold

## Create Fixed Parameters

In [ ]:
fixed_parameters = {
    'expected_wait': 2,
    'gas_required_mean': 15,
    'gas_required_std': 5,
    'pump_rate': 10,
    'expected_lay_time': 5,
    'sim_time': 24 * 60,
}

## Do a test run of the simulation

In [ ]:
gas_sold = sim(
    num_pumps=1,
    **fixed_parameters,
    verbose=True
)

gas_sold

Car 0 arrives at 2.4 minutes
Car 0 begins utilizing a pump at 2.4 minutes
Car 0 leaves its pump at 6.73 minutes
Car 1 arrives at 6.91 minutes
Car 1 begins utilizing a pump at 6.91 minutes
Car 1 leaves its pump at 8.82 minutes
Car 2 arrives at 12.9 minutes
Car 2 begins utilizing a pump at 12.9 minutes
Car 3 arrives at 15.48 minutes
Car 4 arrives at 17.27 minutes
Car 5 arrives at 18.65 minutes
Car 6 arrives at 19.26 minutes
Car 7 arrives at 22.79 minutes
Car 8 arrives at 24.12 minutes
Car 9 arrives at 30.23 minutes
Car 10 arrives at 30.98 minutes
Car 11 arrives at 31.84 minutes
Car 12 arrives at 33.09 minutes
Car 2 leaves its pump at 36.17 minutes
Car 3 begins utilizing a pump at 36.17 minutes
Car 13 arrives at 38.67 minutes
Car 14 arrives at 38.75 minutes
Car 15 arrives at 39.32 minutes
Car 16 arrives at 39.93 minutes
Car 17 arrives at 40.24 minutes
Car 18 arrives at 43.68 minutes
Car 19 arrives at 43.96 minutes
Car 20 arrives at 44.17 minutes
Car 21 arrives at 44.88 minutes
Car 22 arri

Car 574 arrives at 1202.22 minutes
Car 575 arrives at 1203.7 minutes
Car 576 arrives at 1204.21 minutes
Car 577 arrives at 1204.24 minutes
Car 216 leaves its pump at 1204.32 minutes
Car 217 begins utilizing a pump at 1204.32 minutes
Car 578 arrives at 1207.48 minutes
Car 579 arrives at 1209.93 minutes
Car 580 arrives at 1210.24 minutes
Car 217 leaves its pump at 1212.2 minutes
Car 218 begins utilizing a pump at 1212.2 minutes
Car 581 arrives at 1212.96 minutes
Car 218 leaves its pump at 1213.04 minutes
Car 219 begins utilizing a pump at 1213.04 minutes
Car 582 arrives at 1213.46 minutes
Car 583 arrives at 1213.74 minutes
Car 219 leaves its pump at 1214.42 minutes
Car 220 begins utilizing a pump at 1214.42 minutes
Car 584 arrives at 1214.68 minutes
Car 220 leaves its pump at 1217.53 minutes
Car 221 begins utilizing a pump at 1217.53 minutes
Car 585 arrives at 1218.31 minutes
Car 586 arrives at 1220.27 minutes
Car 221 leaves its pump at 1220.36 minutes
Car 222 begins utilizing a pump at 

4032.065610937215

## Do the optimization

In [ ]:
opt_table = pd.Series(
    {num_pumps: sim(num_pumps, **fixed_parameters) for num_pumps in range(1,100)}
)

In [ ]:
arg_max = opt_table.idxmax()
max_gallons_sold = opt_table[arg_max]

print(f'The maximum number of gallons sold is {max_gallons_sold} with {arg_max} pumps')

In [ ]:
opt_table